In [1]:
#Necessary imports for plotting, pandas, math operations, and multiple print statements.
import pandas as pd
import numpy as np

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# read cleaned dataset
df = pd.read_csv("datasets/moviesOFFICIAL.csv")

# get only necessary columns 
main_df = df[["originalTitle", "Scripts", "genres"]]
main_df = main_df[main_df["Scripts"].str.contains("<NA>") == False].reset_index().drop("index", axis=1)
main_df

,originalTitle,Scripts,genres
0,Kate & Leopold,time time proposed fourth dimension yet mortal...,"Comedy,Fantasy,Romance"
1,Barton Fink,kissing goodbye four walls six flights six fli...,"Comedy,Drama,Thriller"
2,Bill & Ted's Bogus Journey,time reached second crucial turning point dest...,"Adventure,Comedy,Fantasy"
3,Billy Bathgate,think makes cheap dago move man took coll man ...,"Crime,Drama,Thriller"
4,The Butcher's Wife,water across sea little island lived young gir...,"Comedy,Fantasy,Romance"
...,...,...,...
2010,Hickok,keep goddamn head right hear take gunner stay ...,Western
2011,Lost in London,venom 5 million poker tourney download go thin...,"Comedy,Drama"
2012,A Bad Moms Christmas,ill blue without ill blue alright white ill bl...,Comedy
2013,SPF-18,real life movies little mixed ask penny cooper...,"Drama,Romance,Sport"


In [3]:
# tf idf 
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy.sparse

tfidf = TfidfVectorizer()
response = tfidf.fit_transform(main_df["Scripts"])
feature_names = tfidf.get_feature_names()
vectorized_df = pd.DataFrame(response.toarray()).set_axis(feature_names, axis=1)
vectorized_df

,00,000,0000,00000,0002,000500010000,001,0010,0012,0013,...,zoology,zoom,zoomed,zooming,zooms,zoos,zounds,zucchini,zwieback,zygote
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2012,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
# samp = vectorized_df.copy()
# samp.insert(0, "movie_titles", main_df["originalTitle"])
# samp = samp.set_index("movie_titles")
# samp.sample(n=5, axis=1).sample(n=5, axis=0)

In [5]:
# edit vectorized dataframe
vectorized_df.insert(0, "movie_genres", main_df["genres"].copy())
vectorized_df.insert(0, "movie_titles", main_df["originalTitle"].copy())

# divide into train and test
vectorized_train = vectorized_df.loc[:1700].copy()
vectorized_test = vectorized_df.loc[1701:].copy()

In [6]:
# splitting genres only for train
vectorized_train["movie_genres"] = vectorized_train["movie_genres"].squeeze().str.split(",")
vectorized_train = vectorized_train.explode("movie_genres")
vectorized_train

,movie_titles,movie_genres,00,000,0000,00000,0002,000500010000,001,0010,...,zoology,zoom,zoomed,zooming,zooms,zoos,zounds,zucchini,zwieback,zygote
0,Kate & Leopold,Comedy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,Kate & Leopold,Fantasy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,Kate & Leopold,Romance,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Barton Fink,Comedy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Barton Fink,Drama,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1699,The Big Wedding,Comedy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1699,The Big Wedding,Drama,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1699,The Big Wedding,Romance,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1700,Seven Psychopaths,Comedy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
# get 250 movies for each genre (Drama, Comedy, Action, Crime, Romance, Adventure, Thriller)
vectorized_train["movie_genres"].value_counts()
sorted_df = vectorized_train.sort_values(by=['movie_genres'], ascending=True)

not_remove = {'Action', 'Adventure', 'Comedy', 'Crime', 'Drama',  'Romance', 'Thriller'}
remove = {'Mystery', 'Fantasy', 'Sci-Fi', 'Biography', 'Animation', 'Family', 'Horror',  'Sport', 'Music', 'War', 'History', 'Musical', 'Western', 'Documentary'}

sorted_df = sorted_df[~sorted_df["movie_genres"].isin(remove)]

vectorized_train = vectorized_train.iloc[0:0]
for x in not_remove:
    temp = sorted_df.loc[sorted_df["movie_genres"] == x]
    vectorized_train = vectorized_train.append(temp.iloc[:270])

# get list of movies used for training (so we dont use them in testing)
train_movies = list(set(vectorized_train["movie_titles"].tolist()))

vectorized_train
vectorized_train["movie_genres"].value_counts()

Drama        972
Comedy       703
Action       473
Crime        410
Romance      367
Adventure    333
Thriller     300
Mystery      176
Fantasy      136
Sci-Fi       126
Biography     91
Animation     88
Family        81
Horror        61
Sport         54
Music         38
War           14
History       13
Musical       12
Western        8
Name: movie_genres, dtype: int64

,movie_titles,movie_genres,00,000,0000,00000,0002,000500010000,001,0010,...,zoology,zoom,zoomed,zooming,zooms,zoos,zounds,zucchini,zwieback,zygote
1060,Killshot,Action,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
840,Ladder 49,Action,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
159,Léon,Action,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1467,The Expendables,Action,0.017967,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1251,The Avengers,Action,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1187,Grace Is Gone,Drama,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
376,Midnight in the Garden of Good and Evil,Drama,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
378,My Best Friend's Wedding,Drama,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1183,Evening,Drama,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Action       270
Thriller     270
Comedy       270
Adventure    270
Romance      270
Crime        270
Drama        270
Name: movie_genres, dtype: int64

In [8]:
# convert "genres" from categorical to quantitative
d = {'Comedy': 0, 'Fantasy': 1, 'Romance': 2, 'Drama': 3, 'Thriller': 4, 'Action': 5, 'Crime': 6, 'Adventure': 7, 
     'Family': 8, 'Mystery': 9, 'Biography': 10, 'Music': 11, 'Horror': 12, 'Sci-Fi': 13, 'Animation': 14,  
     'Sport': 15, 'History': 16,  'Western': 17, 'War': 18,  'Musical': 19, 'Documentary': 20
}

vectorized_train["movie_genres"] = vectorized_train["movie_genres"].map(d)
vectorized_train

,movie_titles,movie_genres,00,000,0000,00000,0002,000500010000,001,0010,...,zoology,zoom,zoomed,zooming,zooms,zoos,zounds,zucchini,zwieback,zygote
1060,Killshot,5,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
840,Ladder 49,5,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
159,Léon,5,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1467,The Expendables,5,0.017967,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1251,The Avengers,5,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1187,Grace Is Gone,3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
376,Midnight in the Garden of Good and Evil,3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
378,My Best Friend's Wedding,3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1183,Evening,3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
# remove movies that are in train set from test set
test_movies = list(set(vectorized_test["movie_titles"].tolist()))
for x in list(set(test_movies).intersection(train_movies)):
    vectorized_test = vectorized_test[vectorized_test["movie_titles"] != x]

vectorized_test

,movie_titles,movie_genres,00,000,0000,00000,0002,000500010000,001,0010,...,zoology,zoom,zoomed,zooming,zooms,zoos,zounds,zucchini,zwieback,zygote
1701,Thanks for Sharing,"Comedy,Drama,Romance",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1702,What Maisie Knew,Drama,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1703,Mud,Drama,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1704,Miss Peregrine's Home for Peculiar Children,"Adventure,Drama,Family",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1705,Carrie,"Drama,Horror",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010,Hickok,Western,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011,Lost in London,"Comedy,Drama",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2012,A Bad Moms Christmas,Comedy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013,SPF-18,"Drama,Romance,Sport",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# convert movie_genres in test data to a list of numerized genres
vectorized_test_temp = vectorized_test.copy()
vectorized_test_temp["movie_genres"] = vectorized_test_temp["movie_genres"].squeeze().str.split(",")
vectorized_test_temp = vectorized_test_temp.explode("movie_genres")
vectorized_test_temp["movie_genres"] = vectorized_test_temp["movie_genres"].map(d)
vectorized_test_temp = vectorized_test_temp.groupby("movie_titles")["movie_genres"].apply(list).reset_index()
vectorized_test_temp
vectorized_test["movie_genres"] = vectorized_test_temp["movie_genres"].values
vectorized_test

,movie_titles,movie_genres
0,3 Days to Kill,"[5, 3, 4]"
1,5 Flights Up,"[0, 3]"
2,A Bad Moms Christmas,[0]
3,A Fighting Man,"[3, 15]"
4,A Little Chaos,"[3, 2]"
...,...,...
307,Wonder Wheel,"[3, 2]"
308,Wonderstruck,"[7, 3, 9]"
309,X-Men: Apocalypse,"[5, 7, 13]"
310,Yoga Hosers,"[5, 0, 1]"


,movie_titles,movie_genres,00,000,0000,00000,0002,000500010000,001,0010,...,zoology,zoom,zoomed,zooming,zooms,zoos,zounds,zucchini,zwieback,zygote
1701,Thanks for Sharing,"[5, 3, 4]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1702,What Maisie Knew,"[0, 3]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1703,Mud,[0],0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1704,Miss Peregrine's Home for Peculiar Children,"[3, 15]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1705,Carrie,"[3, 2]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010,Hickok,"[3, 2]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011,Lost in London,"[7, 3, 9]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2012,A Bad Moms Christmas,"[5, 7, 13]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013,SPF-18,"[5, 0, 1]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# knn 

from sklearn.neighbors import KNeighborsClassifier 
from numpy import dot
from numpy.linalg import norm

X_train = vectorized_train.copy().drop(["movie_titles", "movie_genres"], axis=1)
y_train = vectorized_train["movie_genres"]
X_test = vectorized_test.copy().drop(["movie_titles", "movie_genres"], axis=1)
y_test = vectorized_test["movie_genres"]

# X_train
# X_test
# y_train
# y_test

def cos_sim(x, y):
    return dot(x, y) / (norm(x) * norm(y))

# knn_model = KNeighborsClassifier(n_neighbors=10, weights='distance', metric=cos_sim)
knn_model = KNeighborsClassifier(n_neighbors=15)
knn_model.fit(X=X_train, y=y_train)

KNeighborsClassifier(n_neighbors=15)

In [19]:
test_preds = knn_model.predict(X_test)
test_preds

# printing out neighbors and their distances
# distances, indices = knn_model.kneighbors(X_test)
# distances

array([2, 2, 2, 2, 3, 2, 4, 4, 2, 3, 6, 6, 6, 2, 2, 0, 2, 2, 6, 0, 6, 2,
       2, 2, 2, 6, 4, 6, 0, 7, 4, 3, 4, 7, 5, 2, 6, 6, 2, 0, 4, 2, 2, 2,
       6, 2, 2, 2, 5, 2, 2, 2, 0, 6, 3, 2, 2, 6, 6, 0, 2, 2, 6, 6, 2, 2,
       4, 3, 2, 2, 6, 2, 6, 2, 2, 2, 2, 2, 6, 2, 0, 2, 6, 7, 6, 3, 2, 2,
       0, 0, 6, 2, 6, 0, 3, 5, 5, 2, 7, 3, 2, 5, 4, 0, 2, 6, 2, 2, 2, 3,
       6, 3, 6, 2, 2, 0, 0, 2, 2, 3, 5, 5, 2, 2, 2, 2, 6, 5, 2, 2, 2, 2,
       6, 6, 2, 2, 2, 3, 6, 2, 5, 2, 0, 3, 2, 2, 6, 2, 2, 3, 6, 4, 2, 3,
       7, 3, 2, 6, 2, 5, 2, 6, 6, 0, 2, 2, 6, 6, 6, 2, 2, 2, 6, 2, 6, 2,
       6, 2, 0, 6, 6, 0, 2, 5, 2, 4, 2, 6, 5, 7, 4, 4, 2, 2, 2, 7, 3, 2,
       0, 2, 2, 2, 2, 5, 2, 5, 6, 6, 6, 6, 2, 6, 0, 3, 2, 3, 0, 6, 2, 2,
       4, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 6, 6, 2, 2, 2, 2, 2, 6, 6, 6, 2,
       2, 0, 2, 6, 4, 2, 2, 2, 0, 3, 2, 6, 2, 3, 3, 0, 6, 2, 2, 3, 3, 6,
       2, 6, 2, 0, 0, 6, 2, 2, 2, 2, 4, 0, 6, 4, 2, 2, 2, 2, 6, 3, 2, 2,
       2, 2, 6, 3, 3, 3, 2, 2, 2, 6, 2, 2, 2, 2, 2,

array([[0.8445244 , 0.8445244 , 0.85433808, ..., 0.91394203, 0.92603821,
        0.92603821],
       [0.84432634, 0.87033789, 0.87431783, ..., 0.92830126, 0.92830126,
        0.92911948],
       [1.05243297, 1.05243297, 1.05421204, ..., 1.08305183, 1.08481206,
        1.08481206],
       ...,
       [0.97193453, 0.98217814, 0.99352204, ..., 1.02298211, 1.02298211,
        1.02523536],
       [1.0804633 , 1.10009213, 1.13074609, ..., 1.17610777, 1.17610777,
        1.17643017],
       [1.03695245, 1.22868166, 1.22928808, ..., 1.24165283, 1.24165283,
        1.24190232]])

In [13]:
pd.set_option('display.max_rows', None)
# evaluate the performance of our model

# compute the error percentage of two arrays
# actual is an array of arrays of numbers
# predicted is an array of numbers
# return the number of correct predictions/total
def error_perc(actual, predicted):
    correct = 0;
    for x, y in zip(actual, predicted):
        for i in x:
            if y == i:
                correct = correct + 1
                # print(x, y)
                # print(str(correct) + ", " + str(len(actual)))
    return correct / len(actual)

ep = error_perc(y_test, test_preds)
ep

compare = vectorized_test[["movie_titles","movie_genres"]]
compare = compare.assign(predicted = pd.Series(test_preds).values)
compare


,movie_titles,movie_genres,predicted
1701,Thanks for Sharing,"[5, 3, 4]",2
1702,What Maisie Knew,"[0, 3]",2
1703,Mud,[0],2
1704,Miss Peregrine's Home for Peculiar Children,"[3, 15]",2
1705,Carrie,"[3, 2]",3
1706,The Hunger Games: Catching Fire,"[0, 17]",2
1707,The Hunger Games: Mockingjay - Part 1,"[7, 3, 8]",4
1708,The Hunger Games: Mockingjay - Part 2,"[5, 6, 3]",4
1709,Sex Tape,"[3, 4]",2
1710,Noah,"[7, 8, 1]",3


0.21474358974358973